<a href="https://colab.research.google.com/github/dhy02094/pytorch/blob/master/%EC%9E%91%EB%AC%BC_%EC%9E%8E_%EC%82%AC%EC%A7%84%EC%9C%BC%EB%A1%9C_%EC%A7%88%EB%B3%91_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cd "/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진"


/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진


In [3]:
!unzip -qq "/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진/dataset.zip의 사본"

데이터 분할을 위한 폴더 생성

In [1]:
import os
import shutil

original_dataset_dir = '/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진/dataset'                    # 저장된 데이터 공간
classes_list = os.listdir(original_dataset_dir)       # 저장된 데이터 공간에서 모든 폴더이름 가져오기

base_dir = '/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진/splitted'                               # 나눈 데이터를 앞으로 저장할 폴더
os.mkdir(base_dir)                                    

train_dir = os.path.join(base_dir, 'train')           # base_dir 에 트레인 생성
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for clss in classes_list :                           # 모든 폴더이름을 트레인, 검증, 테스트에 넣기
    os.mkdir(os.path.join(train_dir, clss))
    os.mkdir(os.path.join(validation_dir, clss))
    os.mkdir(os.path.join(test_dir, clss))



데이터 분할과 클래스별 데이터 수 확인

In [2]:
import math

for clss in classes_list :
    path = os.path.join(original_dataset_dir, clss)   #권한을 주는건데 original - clss 로 연결
    fnames = os.listdir(path)      # 경로를 넣으니까 fnames는 저 위를 다 갖는거

# 데이터 개수 분배
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)

    train_fnames = fnames[:train_size]
    print('Train size(',clss,'): ', len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, clss), fname)
        shutil.copyfile(src, dst)

    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print('Validation size(',clss,'): ', len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, clss), fname)
        shutil.copyfile(src, dst)

    test_fnames = fnames[(train_size + validation_size): 
                                (validation_size + train_size + test_size)]
    
    print('Test size(',clss,'): ', len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, clss), fname)
        shutil.copyfile(src, dst)


    

    

Train size( Pepper,_bell___healthy ):  886
Validation size( Pepper,_bell___healthy ):  295
Test size( Pepper,_bell___healthy ):  295
Train size( Grape___Esca_(Black_Measles) ):  829
Validation size( Grape___Esca_(Black_Measles) ):  276
Test size( Grape___Esca_(Black_Measles) ):  276
Train size( Pepper,_bell___Bacterial_spot ):  598
Validation size( Pepper,_bell___Bacterial_spot ):  199
Test size( Pepper,_bell___Bacterial_spot ):  199
Train size( Strawberry___healthy ):  273
Validation size( Strawberry___healthy ):  91
Test size( Strawberry___healthy ):  91
Train size( Grape___Black_rot ):  708
Validation size( Grape___Black_rot ):  236
Test size( Grape___Black_rot ):  236
Train size( Corn___Common_rust ):  715
Validation size( Corn___Common_rust ):  238
Test size( Corn___Common_rust ):  238
Train size( Apple___Apple_scab ):  378
Validation size( Apple___Apple_scab ):  126
Test size( Apple___Apple_scab ):  126
Train size( Potato___healthy ):  91
Validation size( Potato___healthy ):  30


베이스라인 모델 학습을 위한 준비

In [3]:
import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

print('Current cuda device is', DEVICE)

# 오 코랩은 cuda 로 돌리는게 가능하다!

Current cuda device is cuda


In [6]:
BATCH_SIZE = 256
EPOCH = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder   # 계층적인 폴더 구조를 가지고 있는 데이터셋을 불러올때 사용

transform_base = transforms.Compose([transforms.Resize((64,64)),
                                     transforms.ToTensor()])
# transforms.Compose (이미지 전처리 , Augmentation 에 사용)
# augmentation을 통해 좌우 반전, 발긱 조절, 이미지 확대가 가능. 여기선 64,64 로 조정하고 Totensor로 변환하면서 0~1 정규화

train_dataset = ImageFolder(root='./splitted/train',transform = transform_base)

val_dataset = ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/pytorch/작물 잎 사진/splitted/val',
                          transform = transform_base)

from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size = BATCH_SIZE,
                                         shuffle = True,
                                         num_workers = 4) 
# num_workers 는 gpu 얼마나 할당할지.

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
